In [115]:
import numpy as np
import pandas as pd 

In [116]:
df_train = pd.read_csv("../input/credit-dset/train.csv")

/tmp/ipykernel_30/403186247.py:1: DtypeWarning: Columns (26) have mixed types. Specify dtype option on import or set low_memory=False.
  df_train = pd.read_csv("../input/credit-dset/train.csv")


In [117]:
# string -> no of months
def convert_to_months(age_str):
    if pd.isna(age_str):
        return np.nan
    else : 
        parts = age_str.split(' and ')
        years = int(parts[0].split()[0])
        months = int(parts[1].split()[0])
        total_months = (years * 12) + months
        return total_months

In [118]:
loan_type_col = df_train['Loan_Type']
# dropping columns
df_train = df_train.drop(['Name','Loan_Type','ID'], axis=1)
# base salary -> number
df_train['Base_Salary_PerMonth'] = pd.to_numeric(df_train['Base_Salary_PerMonth'],downcast = 'float',errors = 'coerce')
# Total delayed payments (removing underscores if present) -> number
df_train['Total_Delayed_Payments'] = df_train['Total_Delayed_Payments'].str.replace(r'[^-0-9]', '', regex=True)
df_train['Total_Delayed_Payments'] = pd.to_numeric(df_train['Total_Delayed_Payments'],downcast = 'float',errors = 'coerce')
# credit history age -> number (to no of months)
df_train['Credit_History_Age'] = df_train['Credit_History_Age'].apply(convert_to_months)
df_train['Credit_History_Age'] = pd.to_numeric(df_train['Credit_History_Age'],downcast = 'float',errors = 'coerce')
#age -> number
df_train['Age'] = df_train['Age'].str.replace(r'[^-0-9]', '', regex=True)
df_train['Age'] = pd.to_numeric(df_train['Age'],downcast = 'integer',errors = 'coerce')
#Income_annual -> number
df_train['Income_Annual'] = df_train['Income_Annual'].str.replace(r'[^-.0-9]', '', regex=True)
df_train['Income_Annual'] = pd.to_numeric(df_train['Income_Annual'],downcast = 'float',errors = 'coerce')
#Total_Current_Loans -> number
df_train['Total_Current_Loans'] = df_train['Total_Current_Loans'].str.replace(r'[^-0-9]', '', regex=True)
df_train['Total_Current_Loans'] = pd.to_numeric(df_train['Total_Current_Loans'],downcast = 'integer',errors = 'coerce')
#Current_Debt_Outstanding -> number
df_train['Current_Debt_Outstanding'] = df_train['Current_Debt_Outstanding'].str.replace(r'[^-.0-9]', '', regex=True)
df_train['Current_Debt_Outstanding'] = pd.to_numeric(df_train['Current_Debt_Outstanding'],downcast = 'float',errors = 'coerce')
#Credit_Limit -> number
df_train['Credit_Limit'] = pd.to_numeric(df_train['Credit_Limit'],downcast = 'float',errors = 'coerce')
#Monthly_Balance -> number
df_train['Monthly_Balance'] = pd.to_numeric(df_train['Monthly_Balance'],downcast = 'float',errors = 'coerce')

In [119]:
print(df_train.info(),end = "\n\n")
col = "Credit_Limit" #Monthly_Balance
print(df_train[pd.to_numeric(df_train[col],downcast = 'float', errors="coerce").isna()][col].value_counts())
#print(df_train[col].value_counts())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 80000 entries, 0 to 79999
Data columns (total 25 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Customer_ID               80000 non-null  object 
 1   Month                     80000 non-null  object 
 2   Age                       80000 non-null  int16  
 3   Number                    80000 non-null  object 
 4   Profession                80000 non-null  object 
 5   Income_Annual             80000 non-null  float64
 6   Base_Salary_PerMonth      67968 non-null  float32
 7   Total_Bank_Accounts       80000 non-null  int64  
 8   Total_Credit_Cards        80000 non-null  int64  
 9   Rate_Of_Interest          80000 non-null  int64  
 10  Total_Current_Loans       80000 non-null  int16  
 11  Delay_from_due_date       80000 non-null  int64  
 12  Total_Delayed_Payments    74405 non-null  float32
 13  Credit_Limit              78309 non-null  float32
 14  Total_

In [120]:
df_train.drop_duplicates(inplace=True)
print(df_train.isna().sum().to_string())
print(df_train.shape)

Customer_ID                     0
Month                           0
Age                             0
Number                          0
Profession                      0
Income_Annual                   0
Base_Salary_PerMonth        12032
Total_Bank_Accounts             0
Total_Credit_Cards              0
Rate_Of_Interest                0
Total_Current_Loans             0
Delay_from_due_date             0
Total_Delayed_Payments       5595
Credit_Limit                 1691
Total_Credit_Enquiries       1549
Credit_Mix                      0
Current_Debt_Outstanding        0
Ratio_Credit_Utilization        0
Credit_History_Age           7240
Payment_of_Min_Amount           0
Per_Month_EMI                   0
Monthly_Investment           3605
Payment_Behaviour               0
Monthly_Balance               957
Credit_Score                    0
(80000, 25)


In [121]:
unknown_min_pay_repl = df_train['Payment_of_Min_Amount'].mode()[0]
df_train['Payment_of_Min_Amount'] = df_train.groupby("Customer_ID")['Payment_of_Min_Amount'].transform(
    lambda x: x.where(x != "NM", x[x != "NM"].mode().get(0, unknown_min_pay_repl))
)
df_train['Payment_of_Min_Amount'].value_counts()

Payment_of_Min_Amount
Yes    47543
No     32457
Name: count, dtype: int64

In [122]:
unknown_mix_repl = df_train['Credit_Mix'].mode()[0]
df_train['Credit_Mix'] = df_train.groupby("Customer_ID")['Credit_Mix'].transform(
    lambda x: x.where(x != "_", x[x != "_"].mode().get(0, unknown_mix_repl))
)
df_train['Credit_Mix'].value_counts()

Credit_Mix
Standard    36699
Good        24275
Bad         19026
Name: count, dtype: int64

In [123]:
unknown_prof_repl = 'Lawyer'
df_train['Profession'] = df_train.groupby("Customer_ID")['Profession'].transform(
    lambda x: x.where(x != "_______", x[x != "_______"].mode().get(0, unknown_prof_repl))
)
df_train['Profession'].value_counts()

Profession
Lawyer           5692
Engineer         5497
Mechanic         5493
Architect        5479
Scientist        5394
Accountant       5389
Media_Manager    5363
Teacher          5344
Developer        5343
Journalist       5281
Entrepreneur     5235
Doctor           5214
Manager          5172
Musician         5065
Writer           5039
Name: count, dtype: int64

In [124]:
unknown_number_repl = '000-00-0000'
df_train['Number'] = df_train.groupby("Customer_ID")['Number'].transform(
    lambda x: x.where(x != "#F%$D@*&8", x[x != "#F%$D@*&8"].mode().get(0, unknown_number_repl))
)
df_train['Number'].value_counts()

Number
865-33-5347    8
808-71-5249    8
676-82-0055    8
430-15-0729    8
309-02-3479    8
              ..
702-39-1597    2
346-46-1263    2
837-45-0195    2
761-53-5627    2
789-15-7049    2
Name: count, Length: 12500, dtype: int64

In [125]:
unknown_behavior_repl = df_train['Payment_Behaviour'].mode()[0]
df_train['Payment_Behaviour'] = df_train.groupby("Customer_ID")['Payment_Behaviour'].transform(
    lambda x: x.where(x != "!@9#%8", x[x != "!@9#%8"].mode().get(0, unknown_behavior_repl))
)
df_train['Payment_Behaviour'].value_counts()

Payment_Behaviour
Low_spent_Small_value_payments      22144
High_spent_Medium_value_payments    15452
High_spent_Large_value_payments     12375
Low_spent_Medium_value_payments     11724
High_spent_Small_value_payments      9564
Low_spent_Large_value_payments       8741
Name: count, dtype: int64

In [126]:
null_percentages=(df_train.isna().sum()/df_train.shape[0])*100
null_cols = null_percentages.loc[null_percentages > 0]
null_cols

Base_Salary_PerMonth      15.04000
Total_Delayed_Payments     6.99375
Credit_Limit               2.11375
Total_Credit_Enquiries     1.93625
Credit_History_Age         9.05000
Monthly_Investment         4.50625
Monthly_Balance            1.19625
dtype: float64

In [127]:
rows_to_drop = null_cols.loc[null_cols < 5]
df_train.dropna(subset = rows_to_drop.keys(),inplace=True,how='any',axis=0)
print(df_train.isna().sum().to_string())
print(df_train.shape)

Customer_ID                     0
Month                           0
Age                             0
Number                          0
Profession                      0
Income_Annual                   0
Base_Salary_PerMonth        10882
Total_Bank_Accounts             0
Total_Credit_Cards              0
Rate_Of_Interest                0
Total_Current_Loans             0
Delay_from_due_date             0
Total_Delayed_Payments       5094
Credit_Limit                    0
Total_Credit_Enquiries          0
Credit_Mix                      0
Current_Debt_Outstanding        0
Ratio_Credit_Utilization        0
Credit_History_Age           6547
Payment_of_Min_Amount           0
Per_Month_EMI                   0
Monthly_Investment              0
Payment_Behaviour               0
Monthly_Balance                 0
Credit_Score                    0
(72457, 25)


In [128]:
columns_to_drop = null_cols.loc[null_cols > 40]
df_train.drop(columns = columns_to_drop.keys(),inplace = True)
df_train.drop_duplicates(inplace=True)
print(df_train.shape)

(72457, 25)


In [129]:
null_percentages=(df_train.isna().sum()/df_train.shape[0])*100
null_cols = null_percentages.loc[null_percentages > 0]
print(null_cols,end = "\n\n")
col_impute = null_cols.loc[(null_cols >= 5) & (null_cols < 40)]
for column in col_impute.keys():
    central_tend = df_train[column].mean()
    df_train[column] = df_train[column].fillna(central_tend)

Base_Salary_PerMonth      15.018563
Total_Delayed_Payments     7.030377
Credit_History_Age         9.035704
dtype: float64



In [130]:
df_train['Monthly_Investment'] = df_train['Monthly_Investment'].str.replace(r'[^-.0-9]', '', regex=True)
df_train['Monthly_Investment'] = pd.to_numeric(df_train['Monthly_Investment'],downcast = 'float',errors = 'coerce')
df_train = df_train.drop(['Customer_ID'], axis=1)
print(df_train.info())

<class 'pandas.core.frame.DataFrame'>
Index: 72457 entries, 0 to 79999
Data columns (total 24 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Month                     72457 non-null  object 
 1   Age                       72457 non-null  int16  
 2   Number                    72457 non-null  object 
 3   Profession                72457 non-null  object 
 4   Income_Annual             72457 non-null  float64
 5   Base_Salary_PerMonth      72457 non-null  float32
 6   Total_Bank_Accounts       72457 non-null  int64  
 7   Total_Credit_Cards        72457 non-null  int64  
 8   Rate_Of_Interest          72457 non-null  int64  
 9   Total_Current_Loans       72457 non-null  int16  
 10  Delay_from_due_date       72457 non-null  int64  
 11  Total_Delayed_Payments    72457 non-null  float32
 12  Credit_Limit              72457 non-null  float32
 13  Total_Credit_Enquiries    72457 non-null  float64
 14  Credit_Mix 

In [131]:
df_train.to_csv('clean_trained.csv',index = False)